Since I found history.csv to be unusable (for several reasons to do with numpy, as well as bite encoding) I must generate my own testing data here, it is somewhat reduntant but will lead to easier control in further testing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import chess

In [2]:
orig = pd.read_csv('..\\src\\history.csv')

In [3]:
orig.head()

,input,target
0,b'rnbqkbr1/pppppppp/5n2/8/8/7N/PPPPPPPP/RNBQKB...,-40
1,b'rnbqkbr1/pppppppp/5n2/8/8/7N/PPPPPPPP/RNBQKB...,-40
2,b'rnbqkbr1/pppppppp/5n2/8/8/7N/PPPPPPPP/RNBQKB...,-40
3,b'rnbqkbr1/pppppppp/5n2/8/8/7N/PPPPPPPP/RNBQKB...,-40
4,b'rnbqkbr1/pppppppp/5n2/8/8/7N/PPPPPPPP/RNBQKB...,-40


In [4]:
orig.describe()

,target
count,1500.000000
mean,-70.166667
std,29.490994
min,-110.000000
25%,-100.000000
50%,-70.000000
75%,-50.000000
max,0.000000


In [5]:
df = orig.drop_duplicates()

In [6]:
def stringify(enc_str):
    return str(enc_str[2:len(enc_str)-1])

df['input'] = df['input'].apply(stringify)

C:\Users\mmitk\Anaconda3\envs\aienv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [7]:
df.head()

,input,target
0,rnbqkbr1/pppppppp/5n2/8/8/7N/PPPPPPPP/RNBQKB1R...,-40
25,rnbqkbr1/pppppppp/5n2/8/8/5N2/PPPPPPPP/RNBQKB1...,0
50,rnbqkbr1/pppppppp/5n2/8/8/2N5/PPPPPPPP/R1BQKBN...,0
75,rnbqkbr1/pppppppp/5n2/8/8/N7/PPPPPPPP/R1BQKBNR...,-40
100,rnbqkbr1/pppppppp/5n2/8/8/7P/PPPPPPP1/RNBQKBNR...,-50


In [8]:
def bin_matrix_to_decimal(matrix):
    m = matrix.reshape(-1)
    dec = 0
    j = len(m)-1
    for i in range(len(m)-1):
        if m[i]:
            dec += 2**j
        j -= 1
    return float(dec)

In [9]:
def serialize(board):
    import numpy as np
    assert board.is_valid()

    bstate = np.zeros(64, np.uint8)
    for i in range(64):
      pp = board.piece_at(i)
      if pp is not None:
        #print(i, pp.symbol())
        bstate[i] = {"P": 1, "N": 2, "B": 3, "R": 4, "Q": 5, "K": 6, \
                     "p": 9, "n":10, "b":11, "r":12, "q":13, "k": 14}[pp.symbol()]
    if board.has_queenside_castling_rights(chess.WHITE):
      assert bstate[0] == 4
      bstate[0] = 7
    if board.has_kingside_castling_rights(chess.WHITE):
      assert bstate[7] == 4
      bstate[7] = 7
    if board.has_queenside_castling_rights(chess.BLACK):
      assert bstate[56] == 8+4
      bstate[56] = 8+7
    if board.has_kingside_castling_rights(chess.BLACK):
      assert bstate[63] == 8+4
      bstate[63] = 8+7

    if board.ep_square is not None:
      assert bstate[self.board.ep_square] == 0
      bstate[self.board.ep_square] = 8
    bstate = bstate.reshape(8,8)


    state = []
   
    state.append(bin_matrix_to_decimal((bstate>>3)&1))
    state.append(bin_matrix_to_decimal((bstate>>2)&1))
    state.append(bin_matrix_to_decimal((bstate>>1)&1))
    state.append(bin_matrix_to_decimal((bstate>>0)&1))

    state.append(board.turn*1.0)
    
    return state


In [10]:
# to change encoded byte strings above into
# normal string format
def serialize2(board):
    val = serialize(chess.Board(board))
    return val

# apply here
df['input'] = df['input'].apply(serialize2)

C:\Users\mmitk\Anaconda3\envs\aienv\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
df.head()

,input,target
0,"[327678.0, 1.1024811887802974e+19, 1.707765088...",-40
25,"[327678.0, 1.1024811887802974e+19, 1.707765418...",0
50,"[327678.0, 1.1024811887802974e+19, 1.261011414...",0
75,"[327678.0, 1.1024811887802974e+19, 1.261021969...",-40
100,"[327678.0, 1.1024811887802974e+19, 1.722176497...",-50


Changing input column into 5 distinct feature

In [12]:
# serialize and seperate input column into 5 seperate columns
# concatenate result with target column and save back into df
df = pd.concat([df['input'].apply(pd.Series), df['target']], axis = 1)
df.head()

,0,1,2,3,4,target
0,327678.0,1.102481e+19,1.707765e+19,1.311420e+19,1.0,-40
25,327678.0,1.102481e+19,1.707765e+19,1.311420e+19,1.0,0
50,327678.0,1.102481e+19,1.261011e+19,1.311420e+19,1.0,0
75,327678.0,1.102481e+19,1.261022e+19,1.311420e+19,1.0,-40
100,327678.0,1.102481e+19,1.722176e+19,1.311392e+19,1.0,-50


In [13]:
df = df.rename(columns = {0:'b1',1:'b2',2:'b3',3:'b4',4:'b5','target':'eval'})
df.head()

,b1,b2,b3,b4,b5,eval
0,327678.0,1.102481e+19,1.707765e+19,1.311420e+19,1.0,-40
25,327678.0,1.102481e+19,1.707765e+19,1.311420e+19,1.0,0
50,327678.0,1.102481e+19,1.261011e+19,1.311420e+19,1.0,0
75,327678.0,1.102481e+19,1.261022e+19,1.311420e+19,1.0,-40
100,327678.0,1.102481e+19,1.722176e+19,1.311392e+19,1.0,-50


In [14]:
df.to_csv('data.csv', index = False)

Adding random classes

In [15]:
classes = np.random.randint(2, size=len(df.index))
df['class'] = classes

In [16]:
df.pop('eval')
df.head()

,b1,b2,b3,b4,b5,class
0,327678.0,1.102481e+19,1.707765e+19,1.311420e+19,1.0,0
25,327678.0,1.102481e+19,1.707765e+19,1.311420e+19,1.0,1
50,327678.0,1.102481e+19,1.261011e+19,1.311420e+19,1.0,1
75,327678.0,1.102481e+19,1.261022e+19,1.311420e+19,1.0,0
100,327678.0,1.102481e+19,1.722176e+19,1.311392e+19,1.0,0


In [17]:
df.to_csv('test_data1.csv', index = False)